# Experiment with varying alpha parameter

In [1]:
%matplotlib inline
import mpld3
mpld3.enable_notebook()

%load_ext autoreload

In [ ]:
%autoreload
import sys
sys.path.insert(0, "../")

import warnings
import os
import time

import numpy as np
np.random.seed(1)

import matplotlib
import matplotlib.pyplot as plt
import pickle

from optimizers_llc import SGD_llc
from experiment_utils import history_todict, get_val_split
from rotation_rate_utils import get_learning_rate_multipliers, LayerwiseParameterDistanceMemory

from import_task import import_task
from get_training_utils import get_training_schedule, get_stopping_criteria

from keras.callbacks import ModelCheckpoint

In [3]:
def load_results():
    if 'relative_lr_results.p' not in os.listdir('results'):
        return {}
    else:
        with open('results/relative_lr_results.p','rb') as f:
            return pickle.load(f)

def dump_results(results):
    with open('results/relative_lr_results.p','wb') as f:
        pickle.dump(dict(results),f)

def update_results(path, key, value):
    results = load_results()
    position = results
    for p in path:
        position = position[p]
    position.update({key:value})
    dump_results(results)

In [4]:
save_results = True
if not save_results:
    results = {}
monitor_file = 'monitor_relative.txt' 

In [ ]:
for task in ['cifar10','cifar100','tinyImagenet']:
    x_train, y_train, x_test, y_test, get_model, initial_kernels = import_task(task)
    
    if task == 'tinyImagenet':
        [x_train, y_train], [x_val, y_val] = get_val_split(x_train,y_train, 0.1) # validation set needed for early stopping
    else:
        x_val, y_val = x_test, y_test
    
    if save_results:
        results = load_results()
        if task not in results.keys():
            update_results([],task,{})
    elif task not in results.keys():
        results.update({task:{}})
    
    for optimizer in ['llc','SGD','normalized']:
        
        if save_results:
            results = load_results()
            if optimizer not in results[task].keys():
                update_results([task],optimizer,{})
        elif optimizer not in results[task].keys():
            results[task].update({optimizer:{}})
            
        
        alphas = [-0.8, -0.6, -0.4, -0.3, -0.2, -0.1, 0., 0.1, 0.2, 0.3, 0.4, 0.6, 0.8]
        
        for alpha in alphas:
            start = time.time()
            model = get_model()
            
            if optimizer == 'llc':
                lr = 3**-3
            elif optimizer == 'SGD':
                lr = 1. if task == 'cifar10' else 3**-1

            batch_size = 128
            epochs, lr_scheduler = get_training_schedule(task,lr,alpha)
            verbose = 0

            batch_frequency = int((x_train.shape[0]/batch_size))+5 # once per epoch
            lpdm = LayerwiseParameterDistanceMemory(initial_kernels, batch_frequency = batch_frequency)
            
            stop_callback = get_stopping_criteria(task)
            callbacks = [lr_scheduler, lpdm, stop_callback]
            
            multipliers = get_learning_rate_multipliers(model,alpha = alpha)
            metrics = ['accuracy', 'top_k_categorical_accuracy'] if task == 'tinyImagenet' else ['accuracy']
            model.compile(loss='categorical_crossentropy',
                          optimizer=SGD_llc(model,lr=lr, training_mode = optimizer, multipliers = multipliers),
                          metrics=metrics)
            
            if task == 'tinyImagenet':                 
                weights_file = 'best_tinyImagenet_weights.h5'
                callbacks += [ModelCheckpoint(weights_file, monitor='val_acc', save_best_only=True, save_weights_only = True)]
            

            with warnings.catch_warnings():
                warnings.simplefilter("ignore") # removes warning from keras for slow callback
                history = model.fit(x_train,y_train,
                                    epochs = epochs,
                                    batch_size = batch_size,
                                    verbose = verbose,
                                    validation_data = (x_val, y_val),
                                    callbacks = callbacks)
            
            if task == 'tinyImagenet':
                model.load_weights(weights_file)
            
            test_performance = model.evaluate(x_test,y_test, verbose = verbose)
            
            if save_results:
                update_results([task, optimizer],alpha,{'history':history_todict(history),'lpdm':np.array(lpdm.memory),
                                                        'test_performance':test_performance})
            else:
                results[task][optimizer].update({alpha:{'history':history_todict(history),'lpdm':np.array(lpdm.memory),
                                                        'test_performance':test_performance}})
            
            with open(monitor_file,'a') as file:
                file.write(task + ', '+optimizer+', '+str(alpha)+': done in '+str(time.time()-start)+' seconds.\n')